# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 98.3%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,1.00,1.00,1.00,0.98,1.00,0.98,1.00,0.73,1.00,1.00,1.00,1.00,1.00,0.97,1.00,1.00,0.98,1.00,1.00,0.99,0.97,0.99,1.00,1.00,0.98,1.00,0.99,0.97,0.98,1.00,0.70,1.00,0.97,1.00,1.00,1.00,1.00,1.00,0.99,0.99,1.00,0.99,1.00,1.00,1.00,0.97,1.00,1.00,0.97,0.99,1.00,1.00,1.00,1.00,0.98,1.00,0.97,0.99,1.00,1.00,0.98,0.99,1.00,0.98,1.00,1.00,0.99,0.98,0.97,0.99,1.00,1.00,0.99,0.98,1.00,0.99,0.95,1.00,1.00,1.00


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.98


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.99
2,0.98
3,0.98
4,0.98
5,0.99
6,0.99
7,0.99
8,0.99
9,0.99


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.99
     2          0.98
     3          0.98
     4          0.98
     5          0.99
     6          0.99
     7          0.99
     8          0.99
     9          0.99
     10         0.97
     11         0.97
     12         0.96

## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.41%
Gross Capacity Factor: 0.42%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.42,0.42,0.42,0.41,0.42,0.41,0.42,0.28,0.42,0.42,0.42,0.42,0.42,0.41,0.42,0.42,0.40,0.42,0.42,0.42,0.41,0.42,0.42,0.42,0.41,0.42,0.42,0.41,0.41,0.42,0.28,0.42,0.41,0.42,0.42,0.42,0.42,0.42,0.42,0.41,0.42,0.42,0.42,0.42,0.42,0.41,0.42,0.42,0.40,0.42,0.42,0.42,0.42,0.42,0.41,0.42,0.41,0.42,0.42,0.42,0.41,0.42,0.42,0.41,0.42,0.42,0.42,0.41,0.40,0.42,0.42,0.42,0.41,0.40,0.42,0.42,0.40,0.42,0.42,0.42


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.63
2,0.36
3,0.37
4,0.53
5,0.35
6,0.31
7,0.30
8,0.30
9,0.28


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.63
     2          0.36
     3          0.37
     4          0.53
     5          0.35
     6          0.31
     7          0.30
     8          0.30
     9          0.28
     10         0.43
     11         0.54
     12         0.54

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 100%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,1.00


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.92
2,1.07
3,0.96
4,1.05
5,0.98
6,1.03
7,1.00
8,0.95
9,1.03


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

Completion Rate
year month                 
2003 1                 0.92
     2                 1.07
     3                 0.96
     4                 1.05
     5                 0.98
     6                 1.03
     7                 1.00
     8                 0.95
     9                 1.03
     10                0.96
     11                1.04
     12                0.97

## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $43,474.09/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"639,260.42","639,260.42","639,260.42","266,000.00","8,250,000.00"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"10,433,781.25"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"54,760.42","54,760.42","54,760.42",0.00,0.00
2,"49,000.00","49,000.00","49,000.00",0.00,0.00
3,"54,250.00","54,250.00","54,250.00",0.00,0.00
4,"52,500.00","52,500.00","52,500.00",0.00,0.00
5,"54,250.00","54,250.00","54,250.00",0.00,"2,750,000.00"
6,"52,500.00","52,500.00","52,500.00","266,000.00",0.00
7,"54,250.00","54,250.00","54,250.00",0.00,"2,750,000.00"
8,"54,250.00","54,250.00","54,250.00",0.00,0.00
9,"52,500.00","52,500.00","52,500.00",0.00,"2,750,000.00"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00       2,750,000.00  
     6                   52,500.00            266,000.00               0.00  
     7                   54,250.00                  0.00       2,750,000.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00       2,750,000.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00

## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,0.63,0.63,0.62,0.10,0.26


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,0.63,0.63,0.62,0.10,0.26


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [24]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [25]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [26]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00


In [27]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [28]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [29]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"78,750.00"
Repair,"920,549.48"
Crew Transfer,"75,000.00"
Site Travel,0.00
Mobilization,"1,500,000.00"
Weather Delay,"943,101.56"
No Requests,"5,922,593.75"
Not in Shift,"993,786.46"


In [30]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"78,750.00","78,750.00"
Repair,0.00,0.00,0.00,"920,549.48","920,549.48"
Crew Transfer,0.00,0.00,0.00,"75,000.00","75,000.00"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"1,500,000.00","1,500,000.00"
Weather Delay,0.00,0.00,0.00,"943,101.56","943,101.56"
No Requests,0.00,0.00,0.00,"5,922,593.75","5,922,593.75"
Not in Shift,0.00,0.00,0.00,"993,786.46","993,786.46"


In [31]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      78,750.00
     Repair          920,549.48
     Crew Transfer    75,000.00
     Site Travel           0.00
     Mobilization  1,500,000.00
     Weather Delay   943,101.56
     No Requests   5,922,593.75
     Not in Shift    993,786.46

In [32]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                   
1     Maintenance          0.00
      Repair          25,466.15
      Crew Transfer    3,463.54
      Site Travel          0.00
      Mobilization         0.00
      Weather Delay   14,656.25
      No Requests    105,291.67
      Not in Shift    15,403.65
2     Maintenance          0.00
      Repair          23,096.35

In [33]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          25,466.15
           Crew Transfer    3,463.54
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   14,656.25
           No Requests    105,291.67
           Not in Shift    15,403.65
     2     Maintenance          0.00
           Repair          23,096.35
           Crew Transfer    2,916.67
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   13,343.75
           No Requests    101,427.08
           Not in Shift     6,216.15
     3     Maintenance          0.00
           Repair          25,885.42
           Crew Transfer    2,880.21
           Site Travel          0.00

## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [34]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
generator,"4,529,906.25"


In [35]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
generator delay                  0.00              0.00    1,933,606.77   
          maintenance            0.00              0.00       78,750.00   
          repair                 0.00              0.00      920,549.48   

                        total_cost  
component action                    
generator delay       1,933,606.77  
          maintenance    78,750.00  
          repair        920,549.48

In [36]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,generator,"1,522,000.00",0.00,"3,007,906.25","4,529,906.25"


In [37]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,generator,"38,500.00",0.00,"57,458.33","95,958.33"
2,generator,"34,500.00",0.00,"45,572.92","80,072.92"
3,generator,"44,500.00",0.00,"43,677.08","88,177.08"
4,generator,"54,000.00",0.00,"40,177.08","94,177.08"
5,generator,"368,000.00",0.00,"1,263,065.10","1,631,065.10"
6,generator,"130,500.00",0.00,"51,057.29","181,557.29"
7,generator,"351,500.00",0.00,"1,112,656.25","1,464,156.25"
8,generator,"36,500.00",0.00,"37,369.79","73,869.79"
9,generator,"53,000.00",0.00,"50,549.48","103,549.48"


In [38]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     generator       38,500.00              0.00       57,458.33   
     2     generator       34,500.00              0.00       45,572.92   
     3     generator       44,500.00              0.00       43,677.08   
     4     generator       54,000.00              0.00       40,177.08   
     5     generator      368,000.00              0.00    1,263,065.10   
     6     generator      130,500.00              0.00       51,057.29   
     7     generator      351,500.00              0.00    1,112,656.25   
     8     generator       36,500.00              0.00       37,369.79   
     9     generator       53,000.00              0.00       50,549.48   
     10    generator       70,500.00              0.00       94,554.69   
     11    generator      181,500.00              0.00       79,424.48   
     12    generator      159,000.00              0.00      132,343.75   

                       total_cost  
year month component               
2003 1     generator    95,958.33  
     2     generator    80,072.92  
     3     generator    88,177.08  
     4     generator    94,177.08  
     5     generator 1,631,065.10  
     6     generator   181,557.29  
     7     generator 1,464,156.25  
     8     generator    73,869.79  
     9     generator   103,549.48  
     10    generator   165,054.69  
     11    generator   260,924.48  
     12    generator   291,343.75

## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [39]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [40]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"1,600,008.00"


In [41]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


In [42]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


In [43]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [44]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"3,052.00","2,670.50","2,483.00",18.00
major repair,"1,218.24",53.00,29.00,2.00
major replacement,"2,923.47",370.75,"2,923.47",4.00
manual reset,"10,725.84","3,292.00","1,853.25",587.00
medium repair,633.68,564.50,486.00,13.00
minor repair,"4,575.72","2,658.00","1,649.00",206.00


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [45]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"867,355,615.00"


In [46]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"867,355,615.00","11,037,989.50","11,009,405.00","10,975,531.50","10,748,246.00","11,005,265.00","10,740,305.00","11,033,074.50","7,406,357.50","11,004,786.50","11,035,389.50","11,026,747.50","11,016,346.00","11,027,363.50","10,860,918.00","11,012,356.00","11,012,231.00","10,619,149.50","11,004,490.00","11,000,296.50","10,907,812.50","10,747,535.00","10,972,720.50","11,013,899.50","11,032,837.50","10,840,019.50","11,005,818.50","10,970,370.50","10,702,936.00","10,790,608.00","11,032,924.50","7,413,116.50","11,019,018.50","10,681,070.50","11,013,971.50","11,003,895.00","11,014,273.00","11,032,387.00","11,032,231.50","10,952,600.50","10,867,336.50","11,036,852.00","10,960,020.50","11,017,688.00","10,987,490.00","11,023,791.50","10,720,861.50","11,014,949.50","11,001,770.00","10,573,245.50","10,934,730.00","11,005,843.00","11,017,438.50","11,036,730.00","11,033,731.50","10,787,175.50","11,005,606.50","10,835,746.00","10,976,544.00","11,022,569.00","10,993,177.00","10,737,714.00","10,984,958.00","10,988,916.50","10,681,513.00","11,021,399.50","11,000,088.50","10,954,169.50","10,867,163.50","10,550,838.50","10,907,070.00","10,996,346.00","11,005,243.00","10,805,330.00","10,629,369.00","10,984,538.00","10,913,185.50","10,624,780.50","11,025,103.00","11,035,383.00","11,032,916.00"


In [47]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"867,355,615.00"


In [48]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"111,814,998.50"
2,"58,644,405.50"
3,"66,859,816.00"
4,"91,879,910.50"
5,"62,189,819.00"
6,"53,728,091.00"
7,"53,830,541.00"
8,"53,737,661.00"
9,"47,772,657.00"


In [49]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     111,814,998.50
     2      58,644,405.50
     3      66,859,816.00
     4      91,879,910.50
     5      62,189,819.00
     6      53,728,091.00
     7      53,830,541.00
     8      53,737,661.00
     9      47,772,657.00
     10     76,193,678.50
     11     93,956,733.50
     12     96,747,303.50

## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [50]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [51]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [52]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [53]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [54]:
SAM_settings = "SAM_Singleowner_defaults.yaml"
metrics.sam_settings = load_yaml(sim.env.data_dir / "windfarm", SAM_settings)
metrics._setup_pysam()

In [55]:
metrics.pysam_all_outputs()

,Value
Metric,
After Tax NPV ($),"453,690.55"
Nominal LCOE ($/kW),0.18
Real LCOE ($/kW),0.18
After Tax IRR (%),11.00


In [56]:
sim.env.cleanup_log_files(log_only=False)